In [1]:
import requests as r
import psycopg2
from datetime import datetime
import json
import boto3
import pandas as pd
from pestomo_utils import aws_credentials
from envia_email_utils import send_email

In [37]:
HOSTNAME = 'localhost'
USERNAME = 'postgres'
PASSWORD = 'postgres'
DATABASE_NAME = 'prestomo-db'
AWS_BUCKET = 'projeto-aplicado-landing'

session = boto3.Session(profile_name='guilherme')
s3_session = session.resource('s3')
s3_client = session.client("s3")

# def get_stock_price(stock):
#     _full_endpoint_api = API_URL+stock
#     result = r.get(_full_endpoint_api)
#     json_result = result.json()
#     stock_price = 0
#     if 'results' in json_result:
#         stock_price = json_result['results'][stock]['price']
    
#     dict_return = {
#         "price": stock_price
#     }
#     return dict_return

def load_today_acoes_df():
    _today_date = datetime.today().strftime('%d-%m-%Y')
    bucket_scripts_object = s3_session.Bucket(AWS_BUCKET)
    paginator = s3_client.get_paginator("list_objects")
    page_iterator = paginator.paginate(Bucket=AWS_BUCKET, Prefix=f'acoes/{_today_date}')
    
    storage_options={
        "key": aws_credentials()['key'],
        "secret": aws_credentials()['secret'],
    }

    files_to_load = []
    for page in page_iterator:
        if "Contents" in page:
            for obj in page["Contents"]:
                if obj['Size'] > 0:
                    files_to_load.append(obj['Key'])

    dfs = []
    for file in files_to_load:
        df = pd.read_json(f"s3://{AWS_BUCKET}/{file}", storage_options=storage_options, lines=True)
        dfs.append(df)

    df_final = pd.concat(dfs, axis=0)
    return df_final

def get_prices_from_alerts(df):
    dict_result = {}
    _today_date = datetime.today().strftime('%d/%m/%Y')
    float_formatter = "{:_.2f}".format
    
    def feed_dict_result(email, nome, sigla, tipo, setado=0.00, fechado=0.00):
        if email not in dict_result:
            dict_usuario = {
                "nome": nome,
                "data": _today_date,
                "alertas": []
            }
            dict_result[email] = dict_usuario
        
        dict_alerta = {
            "sigla": sigla,
            "tipo": tipo,
            "setado": float_formatter(setado).replace('.',',').replace('_','.'),
            "fechado": float_formatter(fechado).replace('.',',').replace('_','.')
        }
        dict_result[email]['alertas'].append(dict_alerta)
    
    with psycopg2.connect( host=HOSTNAME, user=USERNAME, password=PASSWORD, dbname=DATABASE_NAME) as db_connection:
        cur = db_connection.cursor()
        cur.execute("""
            SELECT a.usuario_id, 
                   a.acao_sigla, 
                   a.valor_minimo, 
                   a.valor_maximo,
                   u.nome,
                   u.email
              FROM alertas_configurados a join usuarios u on (a.usuario_id = u.id)
             ORDER BY a.usuario_id, a.id
        """)
        resultado = cur.fetchall()
    #     print(resultado)
        for usuario_id, acao_sigla, valor_minimo, valor_maximo, nome, email in resultado:
            preco_fechamento = df[df['sigla'] == acao_sigla]['price'][0]
    #         print(f"acao_sigla {acao_sigla}")
    #         print(f"preco_fechamento {preco_fechamento}")
            if valor_minimo != None:
    #             print(f" >> valor_minimo {valor_minimo}")
                if preco_fechamento < valor_minimo:
    #                 print("Envia email sobre preço mínimo atingido")
                    feed_dict_result(email, nome, acao_sigla, 'minimo', float(valor_minimo), float(preco_fechamento))
    #                 print(f"dict_result {dict_result}")
            if valor_maximo != None:
    #             print(f" >> valor_maximo {valor_maximo}")
                if preco_fechamento > valor_maximo:
    #                 print("Envia email sobre preço máximo atingido")
                    feed_dict_result(email, nome, acao_sigla, 'maximo', float(valor_maximo), float(preco_fechamento))            
    #                 print(f"dict_result {dict_result}")

    return dict_result

df_today_acoes = load_today_acoes_df()
dict_emails = get_prices_from_alerts(df_today_acoes)

In [38]:
dict_emails

{'guid.rosa@gmail.com': {'nome': 'Guilherme da Rosa',
  'data': '08/10/2022',
  'alertas': [{'sigla': 'ITSA4',
    'tipo': 'maximo',
    'setado': '8,67',
    'fechado': '10,23'},
   {'sigla': 'ENGI11',
    'tipo': 'minimo',
    'setado': '46,10',
    'fechado': '43,12'}]}}

In [4]:
# send_email(dict_emails)

In [31]:
valor = 5451546.1

In [32]:
round(valor,2)

5451546.1

In [33]:
import numpy as np

In [34]:
np.around(valor, decimals=2)

5451546.1

In [35]:
float_formatter = "{:_.2f}".format

In [36]:
float_formatter(valor).replace('.',',').replace('_','.')

'5.451.546,10'

In [18]:
"ola bb".replace('bb','')

'ola '